# Adding Docs to the Project Repo

okay so the next item on my list was to add some docs on one repo I was trying to clean up.


So my first step was to add some Markdown files in the repo, Since you know they show up nicely in Github.
(Oh and I found that github also shows the rst format as well)
nothing special only I found that to show images inside a Markdown file we use something like [this](https://stackoverflow.com/questions/41604263/how-do-i-display-local-image-in-markdown) :

![image](files/Users/jzhang/Desktop/Isolated.png)



okay, so the next step I thought was to add a sphinx documentation page for the repo, the way the shphinx developer intended. Still, nothingfancy or autodoc or documentation within code, etc., just adding a doc directory and put a sphinx project there, but I wanted to use the Md files in the repo to be used as the page main conetnt.

So again we encounter the issue of referencing outside the doc. 
Well this link 
https://stackoverflow.com/questions/10199233/can-sphinx-link-to-documents-that-are-not-located-in-directories-below-the-root

seemed relevant. But wait, I needed the same thing for the Markdown.

okay I found this thread seeming to deal with the same problem:

https://stackoverflow.com/questions/46278683/include-my-markdown-readme-into-sphinx


Well I did not read this thread carefully. 

what I understood was to create a readme_link.rst in the same directory of the index.rst

```
Readme File
===========

.. mdinclude:: ../../README.md

```

but trying that I got the error that this mdinclude is not recognized.
ugh! if only it could be a simple job.

Then I thought does sphinx by default support markdowns? what if all my files were Mds, including the index.md (or maybe everything else besides that, how can you put toctree or other directives in that?)



## Sphinx with MD  instead of RST all the way


So I retreated my search into finding out how sphinx works with Mark downs.

Does sphinx support Md natively? well maybe not. So my search lead me to this:

https://www.sphinx-doc.org/en/master/usage/markdown.html



So it appears that I should install a new package 

```
pip install --upgrade recommonmark

```


and then update the 'conf.py' the way it describes.

So I did all that and thought okay maybe now that mdinclude would work?!  wrong! it did not.


But I could create some page with pure Md files, hmm, I guess I did, let me try that again!
yep it does work as long as we include some Md file in the index.rst


But that was not exactly what I wanted to do. you cannot have include directives in the md files.
you cannot also include a md file in a directory above the doc root.

you get this error:

```
/media/hossein/HDD/gdrive/SkillsLearningTraining/tutorials-python/main/sphinx/example_two/source/index.rst:9: WARNING: toctree contains reference to nonexisting document 'content_two'

```

So if we have link.rst file in the root directory, then how can we make it work? again if we use the '.. include:: ../content_two.md' directive it will read it as a 'rst'file.


(side note: [this](https://recommonmark.readthedocs.io/en/latest/auto_structify.html#url-resolver) and the other directives in this page could be relevant.)



## Going Back to the Stack Thread

okay that did not work exatcly as I wished, so I went back to the thread to read the fine lines more carefully.
I found these two  in relvant links: 
https://stackoverflow.com/questions/2471804/using-sphinx-with-markdown-instead-of-rst

https://stackoverflow.com/questions/45967058/render-output-from-markdown-file-inside-rst-file

particularly the last one. So basically I found here that it says that in order two read an MD file inside a rst file you need to use another package: 'm2r'

```
pip install m2r

```
it took me to this [link](https://github.com/miyakogi/m2r#sphinx-integration) for directives on how to install and use the m2r package.

so seems we are supposed to add this 

```
extensions = [
    ...,
    'm2r',
]

# source_suffix = '.rst'
source_suffix = ['.rst', '.md']

```

to the 'conf.py'


So I added that like this:
```

extensions = [
    recommonmark,
    'm2r',
]
```

It made sense now, the '... mdinclude::" directive was actually for this one.
So I build the page like this, I get this strange error:
```
Exception occurred:
  File "/home/hossein/.local/lib/python3.8/site-packages/sphinx/registry.py", line 266, in add_source_parser
    for filetype in parser.supported:
AttributeError: 'str' object has no attribute 'supported'
The full traceback has been saved in /tmp/sphinx-err-15m4i2jw.log, if you want to report the issue to the developers.

```

searching into this I find that [here](https://github.com/sparkfun/Qwiic_Template_Py/issues/1) there is mention of "m2r is incompatible with Sphinx 3.x.x and above #1'


and I see notion of the 'm2r2' [here](https://github.com/sphinx-doc/sphinx/issues/8400) "Sphinx 3.3.0 incompatible with m2r2 plugin #8400"
So I guess maybe I need to upgrade! to m2r2 so  I go with pip installing m2r2 and repeating the same stuff with m2r2.
Now I get this error:

```
Extension error:
source_suffix '.md' is already registered
```

hmm, okay so looks like there is some issue with compatibility of m2r/m2r2 with sphinx 3.3.?!

looking a bit further into this I find [this](https://github.com/sphinx-doc/sphinx/issues/8395) and [this one particularly relevant](https://github.com/CrossNox/m2r2/issues/4)

So Turns out that the issue was that you are not supposed to use the two extensions together:
```

extensions = [
    recommonmark,
    'm2r',
]
```

should be infact 

```

extensions = [
       'm2r',
]
```

Also looks like both 'm2r' and 'm2r2' work fine!


## Publishing the Doc pages on Github

Okay so now I have created the first draft of the project documentation page. How I publish it on Github?

I go similar to the homepage, add a '.nojekyll' file  to the repo and everything is in the docs directory in the root of repo.

Then I go to the Github repo settings and enable the page option and set the source directory to docs ( there are two options: root and docs). But pushing the page it does not work.
It can't find the build files. Well, I think to myself, maybe it's becasue I have defined seperate build and source files in the sphinx project settings that creates an additional level of directories. well, wrong! it's not it!

okay so I found some solutiion [here](https://www.docslikecode.com/articles/github-pages-python-sphinx/) and [here](https://github.com/sphinx-doc/sphinx/issues/3382)

```
In your project repo create two directories, docsrc and docs.
In docsrc, initialize sphinx.
Add docsrc/_build/ to your .gitignore
Add the following to the Makefile that sphinx generated for you under docsrc/Makefile
github:
    @make html
    @cp -a _build/html/. ../docs
    
Then you can run make github from the docsrc directory to generate the docs and move them to where GitHub wants them.

```


okay it bascicallly says that just copy the files from the build into the  doc directory, where github searches for it. 
I don't need a make file for that, but the annoying part is that when I added this to the ened of the Makefile it did not work.
Clearly I am missing something about the make sytanx, the one used in the sphinx was a tiny bit more advanced that I could figure out.
So I did a bit of search on makefile syntax. 

Update: okay seems that the issue was more of the character mess up becasue of copy pasting, that is one more reason to STOP this habbit.

BTW, here is the link good for looking at makefile syntax: https://www.gnu.org/software/make/manual/make.html#Rule-Introduction


so my makefile looks like  this:

```
		
html:
	@$(SPHINXBUILD) -M $@ "$(SOURCEDIR)" "$(BUILDDIR)" $(SPHINXOPTS) $(O)

github:
	@make html
	@cp -a _build/html/. ../docs

```

and I tested and works fine on Github!